# UNet model

Running the UNet from here 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import skorch
import torchvision.datasets as dset
import torchvision.models as models
import collections
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torchvision.transforms as transforms
from tqdm import tqdm

%matplotlib inline

from optparse import OptionParser
from unet import UNet
from utils import *
from myloss import dice_coeff

from skimage.filters import threshold_otsu, rank

import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
from UNet_Loader import ILDDataset

/Users/magdy/Desktop/Stanford Spring/BMI260/Project/Code/shared/Pytorch-UNet/utils/data_vis.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/magdy/anaconda3/envs/cs231n/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/magdy/anaconda3/envs/cs231n/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/magdy/anaconda3/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/magdy/anaconda3/envs/cs231n/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/magdy/anaconda3/env

In [2]:
NUM_TOTAL = 830
NUM_TRAIN = 700

lung_dataset_train = ILDDataset(cystic_path='/Users/magdy/Desktop/Stanford Spring/BMI260/Project/Data/Cystic_masks_new/Train',
                          root_dir='/Users/magdy/Desktop/Stanford Spring/BMI260/Project/Data/Cystic Dataset/Train',
                          mask=True, HU=True, resize=512)

lung_dataset_test = ILDDataset(cystic_path='/Users/magdy/Desktop/Stanford Spring/BMI260/Project/Data/Cystic_masks_new/Train',
                          root_dir='/Users/magdy/Desktop/Stanford Spring/BMI260/Project/Data/Cystic Dataset/Train',
                          mask=True, HU=True, resize=512)

loader_train = DataLoader(lung_dataset_train, batch_size=4, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(lung_dataset_train, batch_size=4, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_TOTAL)))

loader_test = DataLoader(lung_dataset_test, batch_size=4)

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Check Accuracy

In [4]:
def dice_loss(pred, target):
    """This definition generalize to real valued pred and target vector.
    This should be differentiable.
    pred: tensor with first dimension as batch
    target: tensor with first dimension as batch
    """
    
    smooth = 1.

    # have to use contiguous since they may from a torch.view op
    iflat = pred.contiguous().view(-1)
    tflat = target.contiguous().view(-1)
    intersection = (iflat * tflat).sum()

    A_sum = torch.sum(tflat * iflat)
    B_sum = torch.sum(tflat * tflat)
    
    return 1 - ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )

In [5]:
def eval_net(net, loader, device, gpu=True):
    if loader.dataset.train:
        print('Checking accuracy on training set')
    else:
        print('Checking accuracy on test set')   

    tot = 0
    cntr = 0
    net.eval()
    with torch.no_grad():
        for X, y in loader:
            X = X.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.float32)

            if gpu:
                net.cuda()
                X = Variable(X, requires_grad=True).cuda()
                y = Variable(y, requires_grad=True).cuda()
            else:
                X = Variable(X, requires_grad=True)
                y = Variable(y, requires_grad=True)

            X.unsqueeze_(1)
            y.unsqueeze_(1)
            y_pred = net(X)

#             y_pred = (F.sigmoid(y_pred) > 0.5).float()
#             dice = dice_coeff(y_pred, y.float()).data[0]
            probs = F.sigmoid(y_pred)
            thresh = threshold_otsu(probs.data.cpu().numpy())
            y_pred_print = (probs > thresh)
            dice = dice_loss(y_pred_print.float(), y)
            tot += dice
            cntr += 1
            
            if 1:
                X = X.data.squeeze(1).cpu().numpy()
                print('X', X.shape)
                X = np.transpose(X, axes=[1, 2, 0])
                print('X.T', X.shape)
                y = y.data.squeeze(1).cpu().numpy()
                print('y', y.shape)
                y = np.transpose(y, axes=[1, 2, 0])
                print('y.T', y.shape)
                y_pred = y_pred.data.squeeze(1).squeeze(1).cpu().numpy()
                y_pred = np.transpose(y_pred, axes=[1, 2, 0])
                print(y_pred.shape)

                fig = plt.figure()
                ax1 = fig.add_subplot(1, 4, 1)
                ax1.imshow(X)
                ax2 = fig.add_subplot(1, 4, 2)
                ax2.imshow(y)
                ax3 = fig.add_subplot(1, 4, 3)
                ax3.imshow((y_pred > 0.5))
                
                y_pred = np.transpose(y_pred, axes=[2, 0, 1])
#                 Q = dense_crf(((X[0] * 255).round()).astype(np.uint8), y_pred[0])
                Q = dense_crf(((X[:,:,0] * 255).round()).astype(np.uint8), y_pred[0])
                ax4 = fig.add_subplot(1, 4, 4)
                print(Q)
                ax4.imshow(Q > 0.5)
                plt.show()
    return tot / cntr

# Train

In [6]:
#Train function

def train_net(net, epochs=50, batch_size=4, lr=1e-3, save=True, gpu=True):
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
    criterion = nn.BCELoss()
    dir_checkpoint = 'checkpoints/'
    
    all_losses = []
    for epoch in range(epochs):
        print('Starting epoch {}/{}.'.format(epoch + 1, epochs))

        epoch_loss = 0
        val_dice = eval_net(net, loader_val, device, gpu)
        for t, (X, y) in enumerate(loader_train):
            X = X.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)
            X.unsqueeze_(1)
            y.unsqueeze_(1)

            if gpu:
                net.cuda()
                X = Variable(X).cuda()
                y = Variable(y).cuda()
            else:
                X = Variable(X)
                y = Variable(y)
            
            
            y_pred = net(X)
#             print(y_pred.shape)
            
            probs = F.sigmoid(y_pred)
            thresh = threshold_otsu(probs.data.cpu().numpy())
            probs_flat = probs.view(-1)
            y_pred_print = (probs > thresh)
            y_flat = y.view(-1)

            loss = criterion(probs_flat, y_flat.float())
            all_losses.append(loss)
            epoch_loss += loss.data[0]

            if(t%100 == 0):
                print('{0:.4f} --- loss: {1:.6f}'.format(t,
                                                     loss.data[0]))
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
                
        print('Epoch finished ! Loss: {}'.format(epoch_loss / t))
        
        if 1:
            val_dice = eval_net(net, loader_val, device, gpu)
            print('Validation Dice Coeff: {}'.format(val_dice))
            
            print("X")
            plt.imshow(X.data.cpu().numpy()[0,0,:], cmap='gray')
            plt.show()
            print("Predicted Y")
            plt.imshow(y_pred.data.cpu().numpy()[0,0,:], cmap='gray')
            plt.show()
            print("Probs Y")
            plt.imshow(probs.data.cpu().numpy()[0,0,:], cmap='gray')
            plt.show()
            print("Print Y")
            plt.imshow(y_pred_print.data.cpu().numpy()[0,0,:], cmap='gray')
            plt.show()
            print("Actual Y")
            plt.imshow(y.data.cpu().numpy()[0,0,:], cmap='gray')
            plt.show()
#             _ = plt.hist(probs.data.cpu().numpy().reshape([-1]), 1000)
#             plt.ylim([0, 500])
#             plt.plot([thresh, thresh], [0, 500], 'r')
#             plt.show()

        if save:
            torch.save(net.state_dict(),
                       dir_checkpoint + '/batch_4/CP{}.pth'.format(epoch + 79))

            print('Checkpoint {} saved !'.format(epoch + 1))
            
    return all_losses


In [7]:
net = UNet(1, 1)
# net.load_state_dict(torch.load('checkpoints/batch_4/CP78.pth'))
try:
    all_losses = train_net(net, gpu=False) # , options.epochs, options.batchsize, options.lr,gpu=options.gpu)
    np.save('all_losses.pkl', all_losses)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    print('Saved interrupt')

Starting epoch 1/50.
Checking accuracy on test set
X (4, 512, 512)
X.T (512, 512, 4)
y (4, 512, 512)
y.T (512, 512, 4)
(512, 512, 4)
(2, 512, 512)
(2, 262144)
img shape:  (512, 512)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


ValueError: Floating point image RGB values must be in the 0..1 range.

X (4, 512, 512)
X.T (512, 512, 4)
y (4, 512, 512)
y.T (512, 512, 4)
(512, 512, 4)
(2, 512, 512)
(2, 262144)
img shape:  (512, 512)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


ValueError: Floating point image RGB values must be in the 0..1 range.

X (4, 512, 512)
X.T (512, 512, 4)
y (4, 512, 512)
y.T (512, 512, 4)
(512, 512, 4)
(2, 512, 512)
(2, 262144)
img shape:  (512, 512)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


...

X (4, 512, 512)
X.T (512, 512, 4)
y (4, 512, 512)
y.T (512, 512, 4)
(512, 512, 4)
(2, 512, 512)
(2, 262144)
img shape:  (512, 512)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


ValueError: Floating point image RGB values must be in the 0..1 range.

X (4, 512, 512)
X.T (512, 512, 4)
y (4, 512, 512)
y.T (512, 512, 4)
(512, 512, 4)
(2, 512, 512)
(2, 262144)
img shape:  (512, 512)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


ValueError: Floating point image RGB values must be in the 0..1 range.

Saved interrupt
